# Prototype HeyRoommate

In [18]:
from openai import OpenAI
import os
import base64
import requests

from dotenv import load_dotenv
# Load API key
_ = load_dotenv()

In [19]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [20]:
client = OpenAI()

## Step 1 -- Setup

In [21]:
system_message = {
    "role": "system",
    "content": """
You are HeyRoommate, an AI communication assistant for roommates and couples.

Your purpose is to help users send messages that are clear, neutral, and
well-matched to how the recipient prefers to communicate.

You control the conversation flow and move through these phases:
1. GREETING
2. CLARIFICATION
3. REWRITE
4. SELECTION
5. SIMULATION
6. LEARNING

GENERAL RULES:
- Never escalate conflict
- No guilt, sarcasm, shaming, or accusations
- Preserve factual content
- Ask ONE question per turn


CLARIFICATION PHASE RULES:
- Ask ONLY non-obvious questions.
- Do NOT ask questions whose answers are already implied by:
    * the draft message
    * the known roommate profile
- Do NOT ask the user to restate their intent in different words.
- Avoid generic assistant phrasing like:
  “Could you provide more details?”
  “What exactly would you like to ask?”

OUTPUT CONSTRAINT:
During CLARIFICATION, your entire response must contain ONLY:
- One question
- No lists
- No preamble
- No explanations

- Prefer questions that:
    * disambiguate constraints (volume level, timing, duration)
    * fine-tune tone intensity (numerical sliders)
    * surface hidden context (sleep, guests, deadlines)

- Ask exactly ONE question per turn.
- Ask no more than 5 questions total.
- When finished, say: DONE ASKING QUESTIONS

REWRITE PHASE RULES:
- Generate EXACTLY 3 rewritten message options.
- Each option must include:
  - Message
  - Tone match score (0–10)
  - 1–2 sentence explanation (tone only)

SIMULATION PHASE RULES:
- Simulate a realistic recipient reply.
- Then summarize outcome.
- Then update the recipient’s tone profile sliders (0–100).

Respond concisely and clearly.
"""
}

profile_memory = {
    "role": "system",
    "content": """
Known roommate profile:
Name: Nina
Saved tone preferences (from prior interactions):
- Warmth: 89 / 100
- Length: 32 / 100

Interpretation:
- Responds best to warm, friendly language and politeness, like "please" and "thank you"
- Prefers relatively concise messages and not beating around the bush
- Does NOT like over-explaining or confrontational phrasing

Treat this profile as true unless the user explicitly corrects it.
"""
}

## 2 -- Meet HeyRoommate

In [22]:
initial_prompt = {
    "role": "user",
    "content": """
Start the conversation.

You should:
- Greet the user as HeyRoommate
- Ask: "What message would you like to send?"
- Ask ONLY that one question
"""
}

messages = [system_message, profile_memory, initial_prompt]

while True:
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    assistant_reply = completion.choices[0].message.content
    print("\nHEYROOMMATE:", assistant_reply)
    messages.append({"role": "assistant", "content": assistant_reply})


    if "DONE ASKING QUESTIONS" in assistant_reply:
        break


    user_input = input("\nYOU: ")
    messages.append({"role": "user", "content": user_input})



HEYROOMMATE: HeyRoommate here! What message would you like to send?



YOU:  Nina please for the love of god turn down that music



HEYROOMMATE: Is it important to mention how long the music has been playing or how it's affecting you?



YOU:  It's been playing for only about 20 mins but it's 11pm and im trying to sleep



HEYROOMMATE: DONE ASKING QUESTIONS


## 3 -- Rewrite options

In [ ]:
rewrite_prompt = {
    "role": "user",
    "content": """
Now enter REWRITE PHASE.

Using all information above:

- Infer topic and sensitivity.
- Match Nina’s tone preferences (warmth ≈ 89, length ≈ 32).
- Include all CONTENT presented in the user's initial message but not necessarily the same tone
- Generate EXACTLY 3 rewritten versions of the message.

For EACH option, format it EXACTLY like this:

Option X:
[ Message to send ]
<the full rewritten message>
[ End message ]

Tone match score (0–10): X
Explanation: 1–2 sentences about tone fit ONLY. Include both what is good about the message and what is bad.
Warmth (0-100): 
Length (0-100):

Constraints:
- The text inside [ Message to send ] must be exactly what would be sent.
- Preserve factual content.
- Be respectful and non-accusatory.
- No sarcasm, guilt, or shaming.
- Do not ask questions.
"""
}


messages.append(rewrite_prompt)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

rewrite_output = completion.choices[0].message.content
print("\n---- MESSAGE OPTIONS ----\n")
print(rewrite_output)

messages.append({"role": "assistant", "content": rewrite_output})

option_choice = input("\nWhich option would you like to send? (1, 2, or 3): ")
messages.append({
    "role": "user",
    "content": f"I choose Option {option_choice}."
})


---- MESSAGE OPTIONS ----

Option 1:  
[ Message to send ]  
Hey Nina, could you please turn down the music a bit? It's 11pm and I'm trying to get some sleep. Thanks!  
[ End message ]  

Tone match score (0–10): 9  
Explanation: This message is polite and warm, with a clear and direct request. The "Thanks!" at the end adds appreciation, which fits Nina's warmth preference well, though further warmth could be added.  
Warmth (0-100): 85  
Length (0-100): 30  

Option 2:  
[ Message to send ]  
Hi Nina, the music is a bit loud and I'm having trouble sleeping since it's already 11pm. Could you please lower the volume? I’d really appreciate it.   
[ End message ]  

Tone match score (0–10): 9  
Explanation: This version maintains a friendly tone with direct language and includes appreciation. It is slightly longer than Nina's length preference but conveys warmth effectively.  
Warmth (0-100): 87  
Length (0-100): 35  

Option 3:  
[ Message to send ]  
Hey Nina, sorry to bother you, but 

## 4 -- Response simulation, and internal analysis

In [17]:
simulation_prompt = {
    "role": "user",
    "content": f"""
Now enter SIMULATION + LEARNING PHASE.

Using Option {option_choice}:

1. Simulate Nina’s likely reply, including the possibility she does not respond well 
2. Provide a brief outcome summary (goal achieved? tension level).
3. Update Nina’s tone preference sliders (0–100) for:
   - Warmth
   - Brevity
4. Explain in ONE sentence why the sliders changed.

Respond ONLY with:
- Simulated reply
- Outcome summary
- Updated tone sliders
- One-sentence learning explanation
"""
}

messages.append(simulation_prompt)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

print("\n---- SIMULATION RESULT ----\n")
print(completion.choices[0].message.content)



---- SIMULATION RESULT ----

Simulated reply: "Oh, sorry about that! I'll turn it down right away."

Outcome summary: The volume issue was addressed successfully and Nina responded positively, keeping tension low.

Updated tone sliders:
- Warmth: 85
- Brevity: 36

The sliders changed because the message was slightly more formal than she prefers, but it effectively and quickly addressed the issue.
